In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from io import StringIO

In [2]:
file_1 = 'CNN_01.pdf'
file_2 = '08852123.pdf'
file_3 = '2019_uda.pdf'

방법 1 : pdfminer


In [6]:
def convert_pdf_to_txt(file_name):
    
    resource_manager = PDFResourceManager()
    
    string_io = StringIO()
    codec = 'utf-8'
    la_params = LAParams()
    
    #device = TextConverter(resource_manager, string_io, codec=codec, laparams=la_params)
    device = TextConverter(resource_manager, string_io, laparams=la_params)
    
    file_pointer = open(file_name, 'rb')
    
    interpreter = PDFPageInterpreter(resource_manager, device)
    password = ''
    max_pages=0
    
    caching= True
    
    pagenos = set()
    
    for page in PDFPage.get_pages(file_pointer, pagenos, maxpages=max_pages, password=password,
                                 caching=caching, check_extractable=True): 
        interpreter.process_page(page)
    ext_txt = string_io.getvalue()
    
    file_pointer.close()
    device.close()
    string_io.close()
    
    #print(ext_txt)
    
    return ext_txt

In [7]:
ext_txt = convert_pdf_to_txt(file_2)

In [8]:
f = open("08852123_pdfminer.txt", 'w')
f.write(ext_txt)
f.close()

방법 1 : PDFMINER

In [9]:
def read_pdf_PDFMINER(pdf_file_path):
    """
    pdf_file_path: 'dir/aaa.pdf'로 구성된 path로부터 
    내부의 text 파일을 모두 읽어서 스트링을 리턴함.
    https://pdfminersix.readthedocs.io/en/latest/tutorials/composable.html
    """
    output_string = StringIO()
    with open(pdf_file_path, 'rb') as f:
        parser = PDFParser(f)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    return str(output_string.getvalue())

In [10]:
text_2 = read_pdf_PDFMINER(file_2)

In [11]:
!pwd

/home/hyerin/work/repos/10_PDF_Extraction


In [12]:
f = open("08852123(3).txt", 'w')
f.write(text_2)
f.close()

방법 2 pdftotext

In [14]:
import pdftotext

In [15]:
f = open("./08852123.pdf", 'rb')
fileReader = pdftotext.PDF(f)
f.close()

In [16]:
# print(fileReader[0])

In [17]:
f = open("(4)_pdftotext_08852123.txt", 'w')

for page in fileReader:
#     print(page)
    f.write(page)
    
f.close()

방법 3: PyPDF2 -> 띄어쓰기 제거됨. 수식 제대로 읽지 못함

In [18]:
# importing required modules  
import PyPDF2  
    
# creating a pdf file object  
pdfFileObj = open(file_2, 'rb')  
    
# creating a pdf reader object  
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)  
    
# printing number of pages in pdf file  
print(pdfReader.numPages)  
   
# creating a page object  
pageObj = pdfReader.getPage(0)  
    
# extracting text from page  
# print(pageObj.extractText())  
text_3 = pageObj.extractText()
    
# closing the pdf file object  
pdfFileObj.close()  

8


In [19]:
f = open("08852123_pypdf2.txt", 'w')
f.write(text_3)
f.close()

방법 4 : tika

In [21]:
from tika import parser

In [ ]:
raw = parser.from_file(file_2)
content = raw['content']
# print(content.strip())
text_4 = content.strip()

2022-02-15 13:21:15,724 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2022-02-15 13:21:25,988 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2022-02-15 13:21:27,135 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [ ]:
#print(content.strip())

In [ ]:
f = open("08852123(4).txt", 'w')
f.write(text_4)
f.close()

PDF 테이블 추출

In [ ]:
import tabula
tabula.environment_info()

방법 5 : tika +  BeautifulSoup (페이지단위접근가능) <p>
https://stackoverflow.com/questions/53093531/python-apache-tika-single-page-parser

In [ ]:
from tika import parser
import re

file_1 = 'test_1.pdf'
file_2 = '08852123.pdf'
file_3 = '2019_uda.pdf'

file_name = file_2

# running: java -jar tika-server1.18.jar before executing code below. 
parsedPDF = parser.from_file(file_name)

fulltext = parsedPDF['content']

# metadata_dict = parsedPDF['metadata']
# title = metadata_dict['title']
# author = metadata_dict['Author'] # capturing all the names from lets say 15 pages. Just want it to capture from first page 
# pages = metadata_dict['xmpTPg:NPages']

In [ ]:
# print(parsedPDF)

In [ ]:
print(author)

In [ ]:
def extract_pages(filename):
    raw_xml = parser.from_file(filename, xmlContent=True)
    body = raw_xml['content'].split('<body>')[1].split('</body>')[0]
    
    body = body.replace('- <\p>\n <p>', '-\n')
    body = body.replace('-\n', '')#.replace('\n', " ")#.replace("<p>", "\n").replace("</p>", "\n")#.replace("</p> <p>", "\n \n")
    body = body.replace('\n', ' ')
    body = body.replace('</p>', '</p>\n')

    # 문장 사이에 연결되지 않는 공백 제거
    regex_10 = re.compile('\w\s*[\<][\/][p][\>]\s+[\<][p][\>]\s*[0-9(a-z]') 
    what = regex_10.findall(body)
    for index in range(len(what)):
        body = body.replace(what[index], what[index][0]+" "+what[index][len(what[index])-1])

    # 특수문자 제거
    regex_11 = re.compile('[\<][p][\>][(\d+)(...)(...)(. . .)(&lt;)(&gt;)±Π•−µπλ∈Π∂βρ〈〉∑(⊗)⊗={×ηα|+o·θ∗].+[\<][\/][p][\>]')
    what = regex_11.findall(body)
    for index in range(len(what)):
        body = body.replace(what[index], '')
    
    # 문장이 아닌 구문을 제외
    regex_12 = re.compile('[\<][p][\>].*[\/][p][\>]')
    what = regex_12.findall(body)
    regex_12_2 = re.compile('\w\w[\.]')
    regex_12_3 = re.compile('\w\w[\:]')
    for index in range(len(what)):
        # .으로 끝나는 문장 찾기
        find_2 = regex_12_2.findall(what[index])
        if len(find_2) == 0:  # .으로 끝나는 문장이 없는 경우 삭제
            find_3 = regex_12_3.findall(what[index]) 
            if len(find_3) == 0:  # .으로 끝나지는 않지만 : 으로 끝나는 문장 찾기
                body = body.replace(what[index], "")
    
    # 공백라인 제거
    regex_13 = re.compile('[\>]\s+[\<][p]') 
    what = regex_13.findall(body)
    for index in range(len(what)):
        body = body.replace(what[index], '>\n<p' )               

    body = body.replace("</p>", '').replace("<p>", '').replace("<p />", '')
    
    # html 태그 제거 
    regex_14 = re.compile('[\<].*[\>].*[\<][\/].*[\>]')  #re.compile('[\<][d][i][v].*[d][i][v][\>]')
    what = regex_14.findall(body)
    for index in range(len(what)):
        body = body.replace(what[index], '\n' )
    
    # 제거 되지 않은 html 부분 한번 더 제거
    what = regex_14.findall(body)
    for index in range(len(what)):
        print(what[index])
        body = body.replace(what[index], '\n' )
        
    body = body.replace("</div>", '')
        
    text_pages = body.split("""<div class="page">""")[1:]

    return text_pages

In [ ]:
separated_pages = extract_pages(file_name)
# print(separated_pages[0])

In [ ]:
print(len(separated_pages))

In [ ]:
import os

In [ ]:
new_txt_name = file_name.split(".")[0]
directory = "./"+new_txt_name
try:
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print ('Error: Creating directory. ' +  directory)

# f = open(directory+"/"+new_txt_name+".txt", 'w')
# f.write(separated_pages)
# f.close()

for page_index in range(len(separated_pages)):
    f = open(directory+"/"+new_txt_name+"_"+str(page_index+1)+".txt", 'w')
    f.write(separated_pages[page_index])
    f.close()
